In [22]:
# 네이버금융에서 특정 종목을 검색해보자
# https://finance.naver.com/item/main.nhn?code=068270 <-- 과거 버전
# https://finance.naver.com/item/main.naver?code=068760
# 068270이 셀트리온의 종목코드다
# 다른종목을 보려면 이 코드만 바꾸면 되는 것.

# 셀트리온의 일별 시세 데이터 전부가져오자
# 페이지당 10개씩 표시되는 상황이다
# 첫 페이지는 최신 날짜의 주가이다
# 화면에서 맨뒤를 클릭하면 가장 오래된 날짜의 종가가 표기된다

# 일별 시세 페이지의 맨뒤버튼에 우클릭을 통해 "페이지 소스 보기"하면
# 아래의 주소가 표시된다
# https://finance.naver.com/item/sise_day.naver?code=068760&page=399
# 여기서 페이지와 종목코드를 넣어서 특정 위치로 갈 수 있다


# beautiful soup와 유명 parser 중 하나인 lxml로 크롤링해보자
# pip install beautifulsoup4
# pip install lxml

# bs4에서 가장 중요한 함수는 find_all() , find() 함수일 것이다

# find_all(['검색할 태그'][, class_='클래스 속성값'][, id='아이디 속성값'][, limit=찾을 개수])
# find(['검색할 태그'][, class_='클래스 속성값'][, id='아이디 속성값'])

# find_all 함수는 문서 전체를 대상으로 조건에 맞는 모든 태그를 찾는다
# 아무것도 못 찾을 때엔 find_all은 비어있는 리스트를 반환, find는 None을 반환

In [ ]:
# 네이버 금융이 현재 크롤링을 막아놨는데, 접속자의 http 헤더를 없애버리는 식 인듯 함
# https://ekyoo.tistory.com/15 여기서 해결법을 찾았고
# 여기서 헤더에 넣을 데이터를 확인할 수 있음 (User Agent 확인)
# http://m.avalon.co.kr/check.html

In [28]:
# bs4를 이용해 네이버금융의  셀트리온의 맨 뒤 페이지 숫자를 구해보자
# 결과를 아래 실제 소스와 비교하면서 보자
# view-source:https://finance.naver.com/item/sise_day.naver?code=068760&page=1

""" 네이버에서 이 코드 막아서 아래것으로 전환 함
url = 'https://finance.naver.com/item/sise_day.naver?code=068760&page=1'
with urlopen(url) as doc:
    html = BeautifulSoup(doc, 'lxml')
    pgrr = html.find('td', class_='pgRR')
    #print(pgrr.a['href'])
"""

from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib import request

headers = ('User-Agent','Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36')
url = 'https://finance.naver.com/item/sise_day.naver?code=068760&page=1'
opener = request.build_opener()
opener.addheaders = [headers]
response = opener.open(url)

html = BeautifulSoup(response, 'lxml')
pgrr = html.find('td', class_='pgRR')
print(pgrr.a['href'])

/item/sise_day.naver?code=068760&page=399
